In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [1]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.misc import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter
from utilities.evaluator import Evaluator
from utilities.gate_killer import GateKiller
from ast import literal_eval


In [2]:
np.random.seed(0)
translator = CirqTranslater(4, untouchable_blocks = [0,1])

translator.env_qubits = [2]

simplifier = Simplifier(translator)
etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)


ddb =concatenate_dbs([x_layer_db(translator)]*2 +[z_layer_db(translator)]*2 + [x_layer_db(translator)]*1 + [x_layer_db(translator)]*2 +  [z_layer_db(translator)]*2 )

cnots = list(np.squeeze(np.array([[k,k] for k in range(translator.number_of_cnots)]).reshape([-1,1])))
ddbc = pd.DataFrame([gate_template(k) for k in cnots])

cdb = concatenate_dbs([ddbc, ddb])
circuit, circuit_db = translator.give_circuit(cdb)


dd1  =circuit_db.copy()
dd1["block_id"] = 1
dd1["symbol"] = None
circuit_db = concatenate_dbs([circuit_db, dd1])
c, circuit_db = translator.give_circuit(circuit_db)


dd1  =circuit_db.copy()
dd1["block_id"] = 2
dd1["symbol"] = None
circuit_db = concatenate_dbs([circuit_db, dd1])
c, circuit_db = translator.give_circuit(circuit_db)

simplifier = Simplifier(translator, apply_relatives_to_first = True)
simplifier.apply_relatives_to_first = True
simplifier.relative_rules = [simplifier.rule_1]
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f2fe8028d30>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f2fe8028d30>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f2fe8028d30>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f2fe8028d30>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f2fe8028d30>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f2fe8028d30>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f2fe8028d30>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f2fe8028d30>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 

AttributeError: ojo <bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7f2fe8028d30>>

In [9]:
mdif = max_diff(translator , circuit_db, simplified_db)
print("\n")
print(mdif)x
print(c)
print()
print(ss)



0.8008441522612086
                                                   ┌──┐   ┌──┐                                                                                                                                                                                                                    ┌──┐   ┌──┐                                                                                                                                                                                                                       ┌──┐   ┌──┐                                                                                                                                                                                                                             ┌──┐   ┌──┐
(0, 0): ───@───@───@───@───@───@───X───X─────────────X──────X────────────────────X───X───Rx(th_0)───Rx(th_4)───Rz(th_8)───Rz(th_12)───Rx(th_16)───Rx(th_20)───Rx(th_24)───Rz(th_28)───Rz(th_32)───────────────────────────@───────────@────

In [3]:
len(simplified_db), len(circuit_db)

NameError: name 'simplified_db' is not defined

In [4]:
import pandas as pd
cc = pd.read_csv("testing/data/dcl")
scc = pd.read_csv("testing/data/dcl_prev")
ppp = cc.drop("Unnamed: 0", axis=1)
pppre = scc.drop("Unnamed: 0", axis=1)
ss,ddb = translator.give_circuit(ppp)
sspre,ddbpre = translator.give_circuit(pppre)

warning, repeated symbol while constructing the circuit, see circuut_
  symbol_name th_72
 symbols ['th_72', 'th_73', 'th_74', 'th_75', 'th_76', 'th_77', 'th_78', 'th_79', 'th_80', 'th_81', 'th_82', 'th_83', 'th_84', 'th_85', 'th_86', 'th_87', 'th_88', 'th_89', 'th_90', 'th_91', 'th_92', 'th_93', 'th_94', 'th_95', 'th_96', 'th_97', 'th_98', 'th_99', 'th_100', 'th_101', 'th_102', 'th_103']
circuit_db {0: {'ind': 16, 'symbol': 'th_72', 'param_value': 0.0, 'trainable': True, 'block_id': 2}, 1: {'ind': 17, 'symbol': 'th_73', 'param_value': 0.0, 'trainable': True, 'block_id': 2}, 2: {'ind': 18, 'symbol': 'th_74', 'param_value': 0.0, 'trainable': True, 'block_id': 2}, 3: {'ind': 19, 'symbol': 'th_75', 'param_value': 0.0, 'trainable': True, 'block_id': 2}, 4: {'ind': 12, 'symbol': 'th_76', 'param_value': 3.448296944257913, 'trainable': True, 'block_id': 2}, 5: {'ind': 13, 'symbol': 'th_77', 'param_value': 4.493667318642263, 'trainable': True, 'block_id': 2}, 6: {'ind': 14, 'symbol': 'th_78', 

In [5]:
ppp

,ind,symbol,param_value,trainable,block_id
0,16,th_72,0.000000,True,2
1,17,th_73,0.000000,True,2
2,18,th_74,0.000000,True,2
3,19,th_75,0.000000,True,2
4,12,th_76,3.448297,True,2
5,13,th_77,4.493667,True,2
6,14,th_78,3.787274,True,2
7,15,th_79,3.423602,True,2
8,16,th_80,0.000000,True,2
9,17,th_81,0.000000,True,2


In [6]:
sspre

(0, 0): ───Rx(th_72)───Rz(th_76)───Rx(th_80)───Rz(th_84)───Rx(th_88)───Rz(th_92)───Rx(th_96)───Rz(th_100)───

(0, 1): ───Rx(th_73)───Rz(th_77)───Rx(th_81)───Rz(th_85)───Rx(th_89)───Rz(th_93)───Rx(th_97)───Rz(th_101)───

(0, 2): ───Rx(th_74)───Rz(th_78)───Rx(th_82)───Rz(th_86)───Rx(th_90)───Rz(th_94)───Rx(th_98)───Rz(th_102)───

(0, 3): ───Rx(th_75)───Rz(th_79)───Rx(th_83)───Rz(th_87)───Rx(th_91)───Rz(th_95)───Rx(th_99)───Rz(th_103)───

ss

In [7]:
def rule_5(simplifier, simplified_db, on_qubit_order, gates_on_qubit):
    """
    compile 1-qubit gates into euler rotations.  ABSOLUTE
    """
    simplification = False
    original_db = simplified_db.copy() #this is not the most efficient thing, just a patch
    for q, qubit_gates_path in gates_on_qubit.items():
        if simplification is True:
            break
        for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-2]):

            if simplification is True:
                break
            ind_gate_p1 = qubit_gates_path[order_gate_on_qubit+1]
            ind_gate_p2 = qubit_gates_path[order_gate_on_qubit+2]

            if (check_rot(ind_gate, simplifier.translator) == True) and (check_rot(ind_gate_p1, simplifier.translator) == True) and (check_rot(ind_gate_p2, simplifier.translator) == True):

                type_0 = type_get(ind_gate,simplifier.translator)
                type_1 = type_get(ind_gate_p1,simplifier.translator)
                type_2 = type_get(ind_gate_p2,simplifier.translator)

                if type_0 == type_2:
                    types = [type_0, type_1, type_2]
                    for next_order_gate_on_qubit, ind_gate_next in enumerate(qubit_gates_path[order_gate_on_qubit+3:]):
                        if (check_rot(ind_gate_next, simplifier.translator) == True):# and (next_order_gate_on_qubit < len(qubit_gates_path[order_gate_on_qubit+3:])):
                            types.append(type_get(ind_gate_next, simplifier.translator))
                            simplification=True
                        else:
                            break
                    if simplification == True:
                        indices_to_compile = [on_qubit_order[q][order_gate_on_qubit+k] for k in range(len(types))]
                        simplifier.translator_ = CirqTranslater(n_qubits=2)
                        u_to_compile_db = simplified_db.loc[indices_to_compile]
                        u_to_compile_db["ind"] = simplifier.translator_.n_qubits*type_get(u_to_compile_db["ind"], simplifier.translator) + simplifier.translator_.number_of_cnots
                        u_to_compile_db["symbol"] = None ##just to be sure it makes no interference with the compiler...

                        compile_circuit, compile_circuit_db = construct_compiling_circuit(simplifier.translator_, u_to_compile_db)
                        minimizer = Minimizer(simplifier.translator_, mode="compiling", hamiltonian="Z")

                        cost, resolver, history = minimizer.minimize([compile_circuit], symbols=simplifier.translator.get_trainable_symbols(compile_circuit_db))

                        OneQbit_translator = CirqTranslater(n_qubits=1)
                        u1s = u1_db(OneQbit_translator, 0, params=True)
                        u1s["param_value"] = -np.array(list(resolver.values()))
                        resu_comp, resu_db = OneQbit_translator.give_circuit(u1s,unresolved=False)


                        u_to_compile_db_1q = u_to_compile_db.copy()
                        u_to_compile_db_1q["ind"] = u_to_compile_db["ind"] = type_get(u_to_compile_db["ind"], simplifier.translator_)

                        cc, cdb = OneQbit_translator.give_circuit(u_to_compile_db_1q, unresolved=False)
                        c = cc.unitary()
                        r = resu_comp.unitary()

                        ## phase_shift if necessary
                        if np.abs(np.mean(c/r) -1) > 1:
                            u1s.loc[0] = u1s.loc[0].replace(to_replace=u1s["param_value"][0], value=u1s["param_value"][0] + 2*np.pi)# Rz(\th) = e^{-ii \theta \sigma_z / 2}c0, cdb0 = simplifier.translator.give_circuit(pd.DataFrame([gate_template(0, param_value=2*np.pi)]), unresolved=False)
                        resu_comp, resu_db = simplifier.translator.give_circuit(u1s,unresolved=False)

                        first_symbols = simplified_db["symbol"][indices_to_compile][:3]

                        for new_ind, typ, pval in zip(indices_to_compile[:3],[0,1,0], list(u1s["param_value"])):
                            simplified_db.loc[new_ind+0.1] = gate_template(simplifier.translator.number_of_cnots + q + typ*simplifier.translator.n_qubits,
                                                                             param_value=pval, block_id=simplified_db.loc[new_ind]["block_id"],
                                                                             trainable=True, symbol=first_symbols[new_ind])

                        for old_inds in indices_to_compile:
                            simplified_db = simplified_db.drop(labels=[old_inds],axis=0)#

                        simplified_db = simplified_db.sort_index().reset_index(drop=True)
                        killed_indices = indices_to_compile[3:]
                        db_follows = original_db[original_db.index>indices_to_compile[-1]]

                        if len(db_follows)>0:
                            gates_to_lower = list(db_follows.index)
                            number_of_shifts = len(killed_indices)
                            for k in range(number_of_shifts):
                                simplified_db = shift_symbols_down(simplifier.translator, gates_to_lower[0]-number_of_shifts, simplified_db)
    return simplification, simplified_db


In [8]:
simplified_db = ddbpre.copy()
ss, simplified_db = translator.give_circuit(ddbpre)
gates_on_qubit, on_qubit_order = simplifier.get_positional_dbs(ss, simplified_db)

In [9]:
simplification = False
original_db = simplified_db.copy() #this is not the most efficient thing, just a patch
for q, qubit_gates_path in gates_on_qubit.items():
    if simplification is True:
        break
    for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-2]):

        if simplification is True:
            break
        ind_gate_p1 = qubit_gates_path[order_gate_on_qubit+1]
        ind_gate_p2 = qubit_gates_path[order_gate_on_qubit+2]

        if (check_rot(ind_gate, simplifier.translator) == True) and (check_rot(ind_gate_p1, simplifier.translator) == True) and (check_rot(ind_gate_p2, simplifier.translator) == True):

            type_0 = type_get(ind_gate,simplifier.translator)
            type_1 = type_get(ind_gate_p1,simplifier.translator)
            type_2 = type_get(ind_gate_p2,simplifier.translator)

            if type_0 == type_2:
                types = [type_0, type_1, type_2]
                for next_order_gate_on_qubit, ind_gate_next in enumerate(qubit_gates_path[order_gate_on_qubit+3:]):
                    if (check_rot(ind_gate_next, simplifier.translator) == True):# and (next_order_gate_on_qubit < len(qubit_gates_path[order_gate_on_qubit+3:])):
                        types.append(type_get(ind_gate_next, simplifier.translator))
                        simplification=True
                    else:
                        break
                if simplification == True:
                    indices_to_compile = [on_qubit_order[q][order_gate_on_qubit+k] for k in range(len(types))]
                    simplifier.translator_ = CirqTranslater(n_qubits=2)
                    u_to_compile_db = simplified_db.loc[indices_to_compile]
                    u_to_compile_db["ind"] = simplifier.translator_.n_qubits*type_get(u_to_compile_db["ind"], simplifier.translator) + simplifier.translator_.number_of_cnots
                    u_to_compile_db["symbol"] = None ##just to be sure it makes no interference with the compiler...

                    compile_circuit, compile_circuit_db = construct_compiling_circuit(simplifier.translator_, u_to_compile_db)
                    minimizer = Minimizer(simplifier.translator_, mode="compiling", hamiltonian="Z")

                    cost, resolver, history = minimizer.minimize([compile_circuit], symbols=simplifier.translator.get_trainable_symbols(compile_circuit_db))

                    OneQbit_translator = CirqTranslater(n_qubits=1)
                    u1s = u1_db(OneQbit_translator, 0, params=True)
                    u1s["param_value"] = -np.array(list(resolver.values()))
                    resu_comp, resu_db = OneQbit_translator.give_circuit(u1s,unresolved=False)


                    u_to_compile_db_1q = u_to_compile_db.copy()
                    u_to_compile_db_1q["ind"] = u_to_compile_db["ind"] = type_get(u_to_compile_db["ind"], simplifier.translator_)

                    cc, cdb = OneQbit_translator.give_circuit(u_to_compile_db_1q, unresolved=False)
                    c = cc.unitary()
                    r = resu_comp.unitary()

                    ## phase_shift if necessary
                    if np.abs(np.mean(c/r) -1) > 1:
                        u1s.loc[0] = u1s.loc[0].replace(to_replace=u1s["param_value"][0], value=u1s["param_value"][0] + 2*np.pi)# Rz(\th) = e^{-ii \theta \sigma_z / 2}c0, cdb0 = simplifier.translator.give_circuit(pd.DataFrame([gate_template(0, param_value=2*np.pi)]), unresolved=False)
                    resu_comp, resu_db = simplifier.translator.give_circuit(u1s,unresolved=False)

                    first_symbols = simplified_db["symbol"][indices_to_compile][:3]

                    for new_ind, typ, pval in zip(indices_to_compile[:3],[0,1,0], list(u1s["param_value"])):
                        simplified_db.loc[new_ind+0.1] = gate_template(simplifier.translator.number_of_cnots + q + typ*simplifier.translator.n_qubits,
                                                                         param_value=pval, block_id=simplified_db.loc[new_ind]["block_id"],
                                                                         trainable=True, symbol=first_symbols[new_ind])

                    for old_inds in indices_to_compile:
                        simplified_db = simplified_db.drop(labels=[old_inds],axis=0)#

                    simplified_db = simplified_db.sort_index().reset_index(drop=True)
                    killed_indices = indices_to_compile[3:]
                    db_follows = original_db[original_db.index>indices_to_compile[-1]]

                    break

In [10]:
db_follows

,ind,symbol,param_value,trainable,block_id
29,13,th_101,4.058272,True,2
30,14,th_102,2.749442,True,2
31,15,th_103,5.603175,True,2


In [ ]:
if len(db_follows)>0:
    print("guacala")
    gates_to_lower = list(db_follows.index)
    number_of_shifts = len(killed_indices)
    for k in range(number_of_shifts):
        simplified_db = shift_symbols_down(simplifier.translator, gates_to_lower[0]-number_of_shifts, simplified_db)

In [20]:
check_symbols_ordered(simplifier.order_symbols(simplified_db))

True

In [19]:
simplified_db

,ind,symbol,param_value,trainable,block_id
0,12,th_72,17.684534,True,2
1,17,th_73,0.000000,True,2
2,18,th_74,0.000000,True,2
3,19,th_75,0.000000,True,2
4,16,th_76,12.566330,True,2
5,13,th_77,4.493667,True,2
6,14,th_78,3.787274,True,2
7,15,th_79,3.423602,True,2
8,12,th_80,-43.164055,True,2
9,17,th_81,0.000000,True,2
